In [1]:
import pandas as pd
import numpy as np

import os
from itertools import combinations

from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from preprocessing_pipeline_Quantifying import PreprocessingPipeline
from explanation_collection import explanation_collection
from differential_privacy import DifferentialPrivacy


In [2]:
dataset_id = int(os.environ.get('DATASET_ID'))
numerical_columns = os.environ.get('NUMERICAL_COLUMNS')
numerical_columns = [item.strip() for item in numerical_columns.split(',')]
case_id = int(os.environ.get('CASE_ID'))
epsilon = int(os.environ.get('EPSILON'))
rep = int(os.environ.get('REPEAT'))

path = os.environ.get('OUTPUT_DIRECTORY')

# dataset_id = 891
# numerical_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age', 'Education', 'Income']
# case_id = 0
# epsilon = 0


In [3]:
# fetch dataset 
uci_dataset = fetch_ucirepo(id=dataset_id)
  
# data (as pandas dataframes) 
X = uci_dataset.data.features 
y = uci_dataset.data.targets 

data = pd.concat([X, y], axis=1)

target_col = y.columns[0]

# Select the remaining columns as categorical columns
all_columns = set(X.columns)
categorical_columns = list(all_columns - set(numerical_columns))

pipeline = PreprocessingPipeline()

In [4]:
steps_prior = [
    'drop_missing_values',

    'encode_categorical_variables',
]

X, y = pipeline.preprocess(data, target_col, numerical_columns, categorical_columns, steps_prior)

if dataset_id == 2:
    y = np.where((y == 0) | (y == 1), 0, 1)

Missing Values Dropped: 0 rows removed
Percentage of rows dropped due to missing values: 0.00%
number of categorical columns '14


In [5]:
# Split data into training and test sets
X_o, X_q, y_o, y_q = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert y_o to a pandas Series
y_o_series = pd.Series(y_o, name=target_col)
y_q_series = pd.Series(y_q, name=target_col)

# Reset index of X_o
X_o = X_o.reset_index(drop=True)
X_q = X_q.reset_index(drop=True)

# Combine X and y into a single DataFrame
df = pd.concat([X_o, y_o_series], axis=1)
    
df_q = pd.concat([X_q, y_q_series], axis=1)

In [6]:
dp = DifferentialPrivacy(epsilon)

df_r = df.copy()

if epsilon != 0:
    df_v = dp.apply_differential_privacy(df, numerical_columns, categorical_columns, round_to_int=True)
else:
    df_v = df.copy()

In [7]:
def get_combination(case_id):
    elements = [1, 2, 3, 4]
    all_combinations = []

    # Generate all non-empty combinations of the elements
    for r in range(4, 0, -1):
        all_combinations.extend(combinations(elements, r))

    # Map the case_id to the corresponding combination
    if 0 <= case_id < len(all_combinations):
        return list(all_combinations[case_id])
    else:
        return "Invalid case_id"

In [8]:
steps = ['drop_all_duplicates', 'handle_outliers', 'scale_numerical_features_only', 'resample_data']

steps_in_r = get_combination(case_id)
steps_in_v = get_combination(case_id)

steps_r = [steps[i-1] for i in steps_in_r]
steps_v = [steps[i-1] for i in steps_in_v]

steps_qr = None
steps_qv = None

if 3 in steps_in_r:
    steps_qr = [steps[2]]
if 3 in steps_in_v:
    steps_qv = [steps[2]]

In [9]:
# Apply the preprocessing pipeline
if steps_r:
    X_r, y_r = pipeline.preprocess(df_r, target_col, numerical_columns, categorical_columns, steps_r, train=True)
else:
    X_r, y_r = df.iloc[:,0:-1], df.iloc[:, -1]

if steps_v:
    X_v, y_v = pipeline.preprocess(df_v, target_col, numerical_columns, categorical_columns, steps_v, train=True)
else:
    X_v, y_v = df.iloc[:,0:-1], df.iloc[:, -1]

# # For querying
# if steps_qr:
#     X_qr, y_qr = pipeline.preprocess(df_q, target_col, numerical_columns, categorical_columns, steps_qr, train=False, X_train=X_r)
# else:
#     X_qr, y_qr = df_q.iloc[:, 0:-1], df_q.iloc[:, -1]

# if steps_qv:
#     X_qv, y_qv = pipeline.preprocess(df_q, target_col, numerical_columns, categorical_columns, steps_qv, train=False, X_train=X_v)
# else:
#     X_qv, y_qv = df_q.iloc[:, 0:-1], df_q.iloc[:, -1]

Duplicates Dropped: 17359 rows removed
Percentage of rows dropped due to duplicates: 8.55%
Outliers: 15233 rows removed
Percentage of rows dropped due to outliers: 8.21%
number of numerical columns '7
Mean relative change for each feature:
 BMI          0.970887
GenHlth      0.780531
MentHlth     0.513143
PhysHlth     0.599856
Age          0.935646
Education    0.858547
Income       0.911513
dtype: float64
Overall mean relative change: 0.7957321294793795
Percentage change in row count due to resampling: 71.44%
Duplicates Dropped: 17359 rows removed
Percentage of rows dropped due to duplicates: 8.55%
Outliers: 15233 rows removed
Percentage of rows dropped due to outliers: 8.21%
number of numerical columns '7
Mean relative change for each feature:
 BMI          0.970887
GenHlth      0.780531
MentHlth     0.513143
PhysHlth     0.599856
Age          0.935646
Education    0.858547
Income       0.911513
dtype: float64
Overall mean relative change: 0.7957321294793795
Percentage change in row 

In [ ]:
# model_r = LogisticRegression(max_iter=1000, random_state=42)
# model_r.fit(X_r, y_r)
# y_pred_r = model_v.predict(X_qr)

In [ ]:
# model_v = LogisticRegression(max_iter=1000, random_state=42)
# model_v.fit(X_v, y_v)
# y_pred_v = model_v.predict(X_qv)

In [ ]:
def pad_or_truncate(data1, data2):
    if isinstance(data1, pd.DataFrame):
        data1 = data1.values
    if isinstance(data2, pd.DataFrame):
        data2 = data2.values

    rows1, cols1 = data1.shape
    rows2, cols2 = data2.shape
    
    # Ensure both datasets have the same number of columns
    if cols1 != cols2:
        raise ValueError("Datasets must have the same number of columns")

    # Pad or truncate the datasets to have the same number of rows
    if rows1 > rows2:
        padding = np.zeros((rows1 - rows2, cols1))
        data2 = np.vstack([data2, padding])
    elif rows1 < rows2:
        padding = np.zeros((rows2 - rows1, cols1))
        data1 = np.vstack([data1, padding])
    
    return data1, data2

def calculate_relative_euclidean_distance(data1, data2):
    data1, data2 = pad_or_truncate(data1, data2)
    vector1 = data1.flatten()
    vector2 = data2.flatten()
    euclidean_distance = np.linalg.norm(vector1 - vector2)
    
    max_possible_distance = np.linalg.norm(np.max(data1, axis=0) - np.min(data1, axis=0)) * np.sqrt(len(vector1))
    relative_distance = euclidean_distance / max_possible_distance
    
    return relative_distance

In [ ]:
# # Accuracy
# model_r_accuracy = accuracy_score(y_qr, y_pred_r)
# model_v_accuracy = accuracy_score(y_qv, y_pred_v)

# Row Metric
row_metric_r = X_r.shape[0] / X_o.shape[0]
row_metric_v = X_v.shape[0] / X_o.shape[0]

relative_distance = calculate_relative_euclidean_distance(X_r, X_o)


In [ ]:
from sklearn.metrics.pairwise import rbf_kernel

def compute_mmd(X_o, X_r, gamma=1.0):
    K_oo = rbf_kernel(X_o, X_o, gamma=gamma)
    K_rr = rbf_kernel(X_r, X_r, gamma=gamma)
    K_or = rbf_kernel(X_o, X_r, gamma=gamma)

    m = len(X_o)
    n = len(X_r)

    mmd_squared = (1/m**2) * np.sum(K_oo) + (1/n**2) * np.sum(K_rr) - (2/(m*n)) * np.sum(K_or)
    return np.sqrt(mmd_squared)

# Assuming X_o and X_r are your dataframes
X_o_array = X_o.values
X_r_array = X_r.values
X_v_array = X_v.values

batch_size = 1000
if dataset_id == 45:
    batch_size = 1
    
num_batches = len(X_o) // batch_size

MMD_r = []
MMD_v = []

for i in range(num_batches):
    X_o_batch = X_o.sample(n=batch_size, random_state=42).values
    X_r_batch = X_r.sample(n=batch_size, random_state=42).values
    X_v_batch = X_v.sample(n=batch_size, random_state=42).values

    mmd_r = compute_mmd(X_o_batch, X_r_batch, gamma=1.0)
    mmd_v = compute_mmd(X_o_batch, X_v_batch, gamma=1.0)

    MMD_r.append(mmd_r)
    MMD_v.append(mmd_v)


overall_mmd_r = np.mean(MMD_r)
overall_mmd_v = np.mean(MMD_v)

In [ ]:
results = np.array([case_id, overall_mmd_r, row_metric_r, relative_distance]).reshape(1,-1)
results_df = pd.DataFrame(results, columns= ['case_id', 'mmd', 'row_metric', 'relative_distance'])

In [ ]:
results_df.to_csv(f'{path}/dataset{dataset_id}_eps{epsilon}_mmd_case{case_id}.csv', index=False)

In [14]:
# df_explanations_r['case'] = case_id
# df_explanations_r.to_csv(f'{path}/dataset{dataset_id}_eps{epsilon}_resR_case{case_id}_rep{rep}.csv', index=False)

# df_explanations_v['case'] = case_id
# df_explanations_v.to_csv(f'{path}/dataset{dataset_id}_eps{epsilon}_resV_case{case_id}_rep{rep}.csv', index=False)

NameError: name 'path' is not defined